In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from copy import deepcopy

In [3]:
from torch.nn.parallel import DistributedDataParallel as DDP, DataParallel as DP

In [4]:
from torch_ecg_bak.torch_ecg.models.nets import BCEWithLogitsWithClassWeightLoss
from torch_ecg_bak.torch_ecg.utils.utils_nn import default_collate_fn as collate_fn
from torch_ecg_bak.torch_ecg.utils.misc import (
    init_logger, get_date_str, dict_to_str, str2bool,
)
from model import ECG_CRNN_CINC2021
from utils.scoring_metrics import evaluate_scores
from cfg import BaseCfg, TrainCfg, ModelCfg
from dataset import CINC2021

In [5]:
from train import train

In [6]:
TrainCfg.db_dir = "/home/wenh06/Jupyter/data/CinC2021/"

In [7]:
config = deepcopy(TrainCfg)
config.db_dir = "/home/wenh06/Jupyter/data/CinC2021/"

In [8]:
config

{'db_dir': '/home/wenh06/Jupyter/data/CinC2021/',
 'log_dir': '/home/wenh06/Jupyter/wenhao/workspace/cinc2021/log',
 'model_dir': '/home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models',
 'final_model_name': None,
 'checkpoints': '/home/wenh06/Jupyter/wenhao/workspace/cinc2021/checkpoints',
 'keep_checkpoint_max': 20,
 'leads': ['I',
  'II',
  'III',
  'aVR',
  'aVL',
  'aVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'fs': 500,
 'data_format': 'channel_first',
 'special_classes': ['Brady', 'LAD', 'RAD', 'PR', 'LQRSV'],
 'normalize_data': True,
 'train_ratio': 0.8,
 'min_class_weight': 0.5,
 'tranches_for_training': '',
 'tranche_class_weights': {'A': {'AF': 0.7604422604422605,
   'IAVB': 1.2860110803324099,
   'CLBBB': 3.9343220338983054,
   'NSR': 1.011437908496732,
   'PAC': 1.5073051948051948,
   'CRBBB': 0.5},
  'B': {'AF': 0.9901960784313726,
   'AFL': 2.805555555555556,
   'CRBBB': 1.3289473684210529,
   'IAVB': 1.429245283018868,
   'IRBBB': 1.7616279069767442,
  

In [9]:
len(config.classes)

21

In [12]:
from cfg import six_leads, four_leads, three_leads, two_leads

In [13]:
ModelFilename = {
    n: f"{n}_lead_model.pth.tar" for n in [12, 6, 4, 3, 2,]
}

### train 12-lead model

In [ ]:
model_config = deepcopy(ModelCfg.twelve_leads)

In [ ]:
model_config

In [ ]:
model_config.cnn.name = config.cnn_name
model_config.rnn.name = config.rnn_name
model_config.attn.name = config.attn_name

In [ ]:
config.n_leads = 12

config.final_model_name = ModelFilename[12]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger = init_logger(log_dir=config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f"Using device {device}")
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f"with configuration\n{dict_to_str(config)}")

tranches = config.tranches_for_training
if tranches:
    classes = config.tranche_classes[tranches]
else:
    classes = config.classes

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=classes,
    n_leads=config.n_leads,
    config=model_config,
)
model.__DEBUG__ = False

if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
config.batch_size, config.n_epochs

In [ ]:
config.batch_size = 128

In [ ]:
_ = train(
    model=model,
    model_config=model_config,
    config=config,
    device=device,
    logger=logger,
    debug=True,
)

### train 6-lead model

In [ ]:
config.leads = six_leads
config.n_leads = 6
config.final_model_name = ModelFilename[6]

config.batch_size = 128

model_config = deepcopy(ModelCfg.six_leads)
model_config.cnn.name = config.cnn_name
model_config.rnn.name = config.rnn_name
model_config.attn.name = config.attn_name

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

del logger

logger = init_logger(log_dir=config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f"Using device {device}")
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f"with configuration\n{dict_to_str(config)}")

tranches = config.tranches_for_training
if tranches:
    classes = config.tranche_classes[tranches]
else:
    classes = config.classes

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=classes,
    n_leads=config.n_leads,
    config=model_config,
)
model.__DEBUG__ = False

if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
_ = train(
    model=model,
    model_config=model_config,
    config=config,
    device=device,
    logger=logger,
    debug=True,
)

### train 4-lead model

In [ ]:
config.leads = four_leads
config.n_leads = 4
config.final_model_name = ModelFilename[4]

config.batch_size = 256

model_config = deepcopy(ModelCfg.four_leads)
model_config.cnn.name = config.cnn_name
model_config.rnn.name = config.rnn_name
model_config.attn.name = config.attn_name

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

del logger

logger = init_logger(log_dir=config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f"Using device {device}")
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f"with configuration\n{dict_to_str(config)}")

tranches = config.tranches_for_training
if tranches:
    classes = config.tranche_classes[tranches]
else:
    classes = config.classes

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=classes,
    n_leads=config.n_leads,
    config=model_config,
)
model.__DEBUG__ = False

if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
_ = train(
    model=model,
    model_config=model_config,
    config=config,
    device=device,
    logger=logger,
    debug=True,
)

### train 3-lead model

In [14]:
config.leads = three_leads
config.n_leads = 3
config.final_model_name = ModelFilename[3]

config.batch_size = 256

model_config = deepcopy(ModelCfg.three_leads)
model_config.cnn.name = config.cnn_name
model_config.rnn.name = config.rnn_name
model_config.attn.name = config.attn_name

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import logging
logger = logging.getLogger()
logger.handlers = []

logger = init_logger(log_dir=config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f"Using device {device}")
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f"with configuration\n{dict_to_str(config)}")

tranches = config.tranches_for_training
if tranches:
    classes = config.tranche_classes[tranches]
else:
    classes = config.classes

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

log file path: /home/wenh06/Jupyter/wenhao/workspace/cinc2021/log/log_06-26_23-36.txt
levels of c_handler and f_handler are set DEBUG
ECG - INFO - 
********************   Start Training   ********************

ECG - INFO - Using device cuda


<IPython.core.display.Javascript object>

ECG - INFO - Using torch of version 1.8.1+cu111
ECG - INFO - with configuration
{
    "db_dir": "/home/wenh06/Jupyter/data/CinC2021/",
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/log",
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models",
    "final_model_name": "3_lead_model.pth.tar",
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/checkpoints",
    "keep_checkpoint_max": 20,
    "leads": [
        "I",
        "II",
        "V2"
    ],
    "fs": 500,
    "data_format": "channel_first",
    "special_classes": [
        "Brady",
        "LAD",
        "RAD",
        "PR",
        "LQRSV"
    ],
    "normalize_data": True,
    "train_ratio": 0.8,
    "min_class_weight": 0.5,
    "tranches_for_training": "",
    "tranche_class_weights": {
        "A": {
            "AF": 0.7604422604422605,
            "IAVB": 1.2860110803324099,
            "CLBBB": 3.9343220338983054,
            "NSR": 1.011437908496732,
            "PAC"

In [16]:
model = ECG_CRNN_CINC2021(
    classes=classes,
    n_leads=config.n_leads,
    config=model_config,
)
model.__DEBUG__ = False

if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

classes (totally 21) for prediction:['AF', 'AFL', 'BBB', 'CLBBB', 'CRBBB', 'IAVB', 'IRBBB', 'LAnFB', 'LPR', 'LQT', 'NSIVCB', 'NSR', 'PAC', 'PRWP', 'PVC', 'QAb', 'SA', 'SB', 'STach', 'TAb', 'TInv']
configuration of ECG_CRNN_CINC2021 is as follows
{
    "cnn": {
        "name": "multi_scopic_leadwise",
        "vgg16": {
            "num_convs": [
                2, 2, 3, 3, 3
            ],
            "num_filters": [
                48, 96, 192, 384, 384
            ],
            "groups": 1,
            "kernel_initializer": "he_normal",
            "kw_initializer": {},
            "activation": "relu",
            "kw_activation": {},
            "block": {
                "filter_length": 3,
                "subsample_length": 1,
                "dilation": 1,
                "batch_norm": True,
                "pool_size": 3,
                "pool_stride": 2,
                "kernel_initializer": "he_normal",
                "kw_initializer": {},
                "activation": "r

<IPython.core.display.Javascript object>

DataParallel(
  (module): ECG_CRNN_CINC2021(
    (cnn): MultiScopicCNN(
      (branches): ModuleDict(
        (branch_0): MultiScopicBranch(
          (block_0): MultiScopicBasicBlock(
            (ca_0): Conv_Bn_Activation(
              (conv1d): Conv1d(3, 96, kernel_size=(11,), stride=(1,), padding=(5,), groups=3)
              (batch_norm): GroupNorm(3, 96, eps=1e-05, affine=True)
              (activation_relu): ReLU(inplace=True)
            )
            (bn): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (down): DownSample(
              (down_sample): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            )
          )
          (block_1): MultiScopicBasicBlock(
            (ca_0): Conv_Bn_Activation(
              (conv1d): Conv1d(96, 192, kernel_size=(7,), stride=(1,), padding=(3,), groups=3)
              (batch_norm): GroupNorm(3, 192, eps=1e-05, affine=True)
              (activation_relu): ReLU

In [17]:
_ = train(
    model=model,
    model_config=model_config,
    config=config,
    device=device,
    logger=logger,
    debug=True,
)

ECG - INFO - training configurations are as follows:
{
    "db_dir": "/home/wenh06/Jupyter/data/CinC2021/",
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/log",
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models",
    "final_model_name": "3_lead_model.pth.tar",
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/checkpoints",
    "keep_checkpoint_max": 20,
    "leads": [
        "I",
        "II",
        "V2"
    ],
    "fs": 500,
    "data_format": "channel_first",
    "special_classes": [
        "Brady",
        "LAD",
        "RAD",
        "PR",
        "LQRSV"
    ],
    "normalize_data": True,
    "train_ratio": 0.8,
    "min_class_weight": 0.5,
    "tranches_for_training": "",
    "tranche_class_weights": {
        "A": {
            "AF": 0.7604422604422605,
            "IAVB": 1.2860110803324099,
            "CLBBB": 3.9343220338983054,
            "NSR": 1.011437908496732,
            "PAC": 1.5073051948051948,
     


start performing train test split...

train test split finished in 0.02 minutes
log file path is set /home/wenh06/Jupyter/wenhao/workspace/cinc2021/working_dir/CinC2021-CinC2021.log
levels of c_handler and f_handler are set DEBUG

start performing train test split...

train test split finished in 0.02 minutes
log file path is set /home/wenh06/Jupyter/wenhao/workspace/cinc2021/working_dir/CinC2021-CinC2021.log
levels of c_handler and f_handler are set DEBUG

start performing train test split...

train test split finished in 0.02 minutes
ECG - INFO - 
Starting training:
------------------
Epochs:          80
Batch size:      256
Learning rate:   0.001
Training size:   61944
Validation size: 15505
Device:          cuda
Optimizer:       adamw_amsgrad
Dataset classes: ['AF', 'AFL', 'BBB', 'CLBBB', 'CRBBB', 'IAVB', 'IRBBB', 'LAnFB', 'LPR', 'LQT', 'NSIVCB', 'NSR', 'PAC', 'PRWP', 'PVC', 'QAb', 'SA', 'SB', 'STach', 'TAb', 'TInv']
Class weights:   tensor([[ 2.7578,  1.7252, 28.8333,  8.4625,  2

Epoch 1/80:   8%|█▉                       | 4864/61944 [00:21<03:22, 282.08it/s, loss (batch)=0.958]

ECG - INFO - Train step_20: loss : 0.9580796957015991


Epoch 1/80:  16%|████                     | 9984/61944 [00:43<02:38, 328.39it/s, loss (batch)=0.765]

...

ECG - INFO - Train step_7000: loss : 0.47137850522994995


Epoch 29/80: 100%|███████████████████████| 61944/61944 [03:34<00:00, 440.45it/s, loss (batch)=0.471]

ECG - DEBUG - all_scalar_preds.shape = (61944, 21), all_labels.shape = (61944, 21)
ECG - DEBUG - 
----------------------------------------------
scalar prediction:    [0.002, 0.003, 0.557, 0.738, 0.001, 0.0, 0.0, 0.002, 0.006, 0.001, 0.0, 0.301, 0.002, 0.001, 0.025, 0.001, 0.001, 0.014, 0.006, 0.008, 0.006]
binary prediction:    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels:               [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predicted classes:    ['BBB', 'CLBBB']
label classes:        ['BBB', 'CLBBB']
----------------------------------------------

ECG - DEBUG - 
----------------------------------------------
scalar prediction:    [0.008, 0.004, 0.002, 0.005, 0.008, 0.076, 0.001, 0.002, 0.001, 0.001, 0.018, 0.007, 0.022, 0.004, 0.004, 0.009, 0.0, 0.01, 0.818, 0.042, 0.021]
binary prediction:    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
labels:               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

Epoch 29/80: 100%|███████████████████████| 61944/61944 [08:20<00:00, 123.65it/s, loss (batch)=0.471]

ECG - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models/3_lead_model.pth.tar!


In [18]:
with torch.no_grad():
    torch.cuda.empty_cache()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### train 2-lead model

In [19]:
config.leads = two_leads
config.n_leads = 2
config.final_model_name = ModelFilename[2]

config.batch_size = 256

model_config = deepcopy(ModelCfg.two_leads)
model_config.cnn.name = config.cnn_name
model_config.rnn.name = config.rnn_name
model_config.attn.name = config.attn_name

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

del logger

logger = init_logger(log_dir=config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f"Using device {device}")
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f"with configuration\n{dict_to_str(config)}")

tranches = config.tranches_for_training
if tranches:
    classes = config.tranche_classes[tranches]
else:
    classes = config.classes

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

log file path: /home/wenh06/Jupyter/wenhao/workspace/cinc2021/log/log_06-27_03-27.txt
levels of c_handler and f_handler are set DEBUG
ECG - INFO - 
********************   Start Training   ********************

ECG - INFO - 
********************   Start Training   ********************

ECG - INFO - Using device cuda
ECG - INFO - Using device cuda


<IPython.core.display.Javascript object>

ECG - INFO - Using torch of version 1.8.1+cu111
ECG - INFO - Using torch of version 1.8.1+cu111
ECG - INFO - with configuration
{
    "db_dir": "/home/wenh06/Jupyter/data/CinC2021/",
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/log",
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models",
    "final_model_name": "2_lead_model.pth.tar",
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/checkpoints",
    "keep_checkpoint_max": 20,
    "leads": [
        "I",
        "II"
    ],
    "fs": 500,
    "data_format": "channel_first",
    "special_classes": [
        "Brady",
        "LAD",
        "RAD",
        "PR",
        "LQRSV"
    ],
    "normalize_data": True,
    "train_ratio": 0.8,
    "min_class_weight": 0.5,
    "tranches_for_training": "",
    "tranche_class_weights": {
        "A": {
            "AF": 0.7604422604422605,
            "IAVB": 1.2860110803324099,
            "CLBBB": 3.9343220338983054,
            "NSR": 1.

ECG - INFO - with configuration
{
    "db_dir": "/home/wenh06/Jupyter/data/CinC2021/",
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/log",
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models",
    "final_model_name": "2_lead_model.pth.tar",
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/checkpoints",
    "keep_checkpoint_max": 20,
    "leads": [
        "I",
        "II"
    ],
    "fs": 500,
    "data_format": "channel_first",
    "special_classes": [
        "Brady",
        "LAD",
        "RAD",
        "PR",
        "LQRSV"
    ],
    "normalize_data": True,
    "train_ratio": 0.8,
    "min_class_weight": 0.5,
    "tranches_for_training": "",
    "tranche_class_weights": {
        "A": {
            "AF": 0.7604422604422605,
            "IAVB": 1.2860110803324099,
            "CLBBB": 3.9343220338983054,
            "NSR": 1.011437908496732,
            "PAC": 1.5073051948051948,
            "CRBBB": 0.5
        },
    

In [21]:
model = ECG_CRNN_CINC2021(
    classes=classes,
    n_leads=config.n_leads,
    config=model_config,
)
model.__DEBUG__ = False

if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

classes (totally 21) for prediction:['AF', 'AFL', 'BBB', 'CLBBB', 'CRBBB', 'IAVB', 'IRBBB', 'LAnFB', 'LPR', 'LQT', 'NSIVCB', 'NSR', 'PAC', 'PRWP', 'PVC', 'QAb', 'SA', 'SB', 'STach', 'TAb', 'TInv']
configuration of ECG_CRNN_CINC2021 is as follows
{
    "cnn": {
        "name": "multi_scopic_leadwise",
        "vgg16": {
            "num_convs": [
                2, 2, 3, 3, 3
            ],
            "num_filters": [
                48, 96, 192, 384, 384
            ],
            "groups": 1,
            "kernel_initializer": "he_normal",
            "kw_initializer": {},
            "activation": "relu",
            "kw_activation": {},
            "block": {
                "filter_length": 3,
                "subsample_length": 1,
                "dilation": 1,
                "batch_norm": True,
                "pool_size": 3,
                "pool_stride": 2,
                "kernel_initializer": "he_normal",
                "kw_initializer": {},
                "activation": "r

<IPython.core.display.Javascript object>

DataParallel(
  (module): ECG_CRNN_CINC2021(
    (cnn): MultiScopicCNN(
      (branches): ModuleDict(
        (branch_0): MultiScopicBranch(
          (block_0): MultiScopicBasicBlock(
            (ca_0): Conv_Bn_Activation(
              (conv1d): Conv1d(2, 64, kernel_size=(11,), stride=(1,), padding=(5,), groups=2)
              (batch_norm): GroupNorm(2, 64, eps=1e-05, affine=True)
              (activation_relu): ReLU(inplace=True)
            )
            (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (down): DownSample(
              (down_sample): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            )
          )
          (block_1): MultiScopicBasicBlock(
            (ca_0): Conv_Bn_Activation(
              (conv1d): Conv1d(64, 128, kernel_size=(7,), stride=(1,), padding=(3,), groups=2)
              (batch_norm): GroupNorm(2, 128, eps=1e-05, affine=True)
              (activation_relu): ReLU

In [22]:
train(
    model=model,
    model_config=model_config,
    config=config,
    device=device,
    logger=logger,
    debug=True,
)

ECG - INFO - training configurations are as follows:
{
    "db_dir": "/home/wenh06/Jupyter/data/CinC2021/",
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/log",
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models",
    "final_model_name": "2_lead_model.pth.tar",
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/cinc2021/checkpoints",
    "keep_checkpoint_max": 20,
    "leads": [
        "I",
        "II"
    ],
    "fs": 500,
    "data_format": "channel_first",
    "special_classes": [
        "Brady",
        "LAD",
        "RAD",
        "PR",
        "LQRSV"
    ],
    "normalize_data": True,
    "train_ratio": 0.8,
    "min_class_weight": 0.5,
    "tranches_for_training": "",
    "tranche_class_weights": {
        "A": {
            "AF": 0.7604422604422605,
            "IAVB": 1.2860110803324099,
            "CLBBB": 3.9343220338983054,
            "NSR": 1.011437908496732,
            "PAC": 1.5073051948051948,
            "CRBBB"

log file path is set /home/wenh06/Jupyter/wenhao/workspace/cinc2021/working_dir/CinC2021-CinC2021.log
levels of c_handler and f_handler are set DEBUG

start performing train test split...

train test split finished in 0.02 minutes
log file path is set /home/wenh06/Jupyter/wenhao/workspace/cinc2021/working_dir/CinC2021-CinC2021.log
levels of c_handler and f_handler are set DEBUG

start performing train test split...

train test split finished in 0.02 minutes
log file path is set /home/wenh06/Jupyter/wenhao/workspace/cinc2021/working_dir/CinC2021-CinC2021.log
levels of c_handler and f_handler are set DEBUG

start performing train test split...

train test split finished in 0.02 minutes
ECG - INFO - 
Starting training:
------------------
Epochs:          80
Batch size:      256
Learning rate:   0.001
Training size:   61944
Validation size: 15505
Device:          cuda
Optimizer:       adamw_amsgrad
Dataset classes: ['AF', 'AFL', 'BBB', 'CLBBB', 'CRBBB', 'IAVB', 'IRBBB', 'LAnFB', 'LPR', 'LQ

Epoch 1/80:   8%|██                        | 4864/61944 [00:14<02:04, 458.68it/s, loss (batch)=1.33]

ECG - INFO - Train step_20: loss : 1.332281470298767
ECG - INFO - Train step_20: loss : 1.332281470298767


Epoch 1/80:  16%|████                     | 9984/61944 [00:27<01:53, 456.87it/s, loss (batch)=0.785]

...

ECG - DEBUG - all_scalar_preds.shape = (61944, 21), all_labels.shape = (61944, 21)
ECG - DEBUG - all_scalar_preds.shape = (61944, 21), all_labels.shape = (61944, 21)
ECG - DEBUG - 
----------------------------------------------
scalar prediction:    [0.268, 0.267, 0.009, 0.013, 0.065, 0.011, 0.015, 0.01, 0.01, 0.02, 0.057, 0.018, 0.019, 0.002, 0.006, 0.004, 0.002, 0.002, 0.032, 0.083, 0.047]
binary prediction:    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels:               [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predicted classes:    ['AF', 'AFL']
label classes:        ['AF']
----------------------------------------------

ECG - DEBUG - 
----------------------------------------------
scalar prediction:    [0.268, 0.267, 0.009, 0.013, 0.065, 0.011, 0.015, 0.01, 0.01, 0.02, 0.057, 0.018, 0.019, 0.002, 0.006, 0.004, 0.002, 0.002, 0.032, 0.083, 0.047]
binary prediction:    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
label

ECG - DEBUG - 
----------------------------------------------
scalar prediction:    [0.019, 0.007, 0.002, 0.016, 0.007, 0.004, 0.007, 0.012, 0.009, 0.004, 0.003, 0.388, 0.004, 0.011, 0.009, 0.009, 0.961, 0.012, 0.002, 0.001, 0.006]
binary prediction:    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
labels:               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
predicted classes:    ['SA']
label classes:        ['SA']
----------------------------------------------

ECG - DEBUG - 
----------------------------------------------
scalar prediction:    [0.019, 0.007, 0.002, 0.016, 0.007, 0.004, 0.007, 0.012, 0.009, 0.004, 0.003, 0.388, 0.004, 0.011, 0.009, 0.009, 0.961, 0.012, 0.002, 0.001, 0.006]
binary prediction:    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
labels:               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
predicted classes:    ['SA']
label classes:        ['SA']
----------------------------

Epoch 21/80: 100%|███████████████████████| 61944/61944 [07:46<00:00, 132.92it/s, loss (batch)=0.533]

ECG - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models/2_lead_model.pth.tar!
ECG - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/cinc2021/saved_models/2_lead_model.pth.tar!


OrderedDict([('cnn.branches.branch_0.block_0.ca_0.conv1d.weight',
              tensor([[[-5.8933e-02,  1.3481e-01,  5.0512e-01,  2.4454e-01,  9.9470e-02,
                         5.3574e-01,  4.0170e-01,  4.4804e-01,  3.3759e-01,  2.0570e-01,
                        -2.7713e-02]],
              
                      [[-8.6655e-02, -4.1103e-01, -4.6176e-01,  3.0567e-02,  6.1267e-01,
                        -3.3985e-01,  2.4354e-01,  9.8942e-01, -2.6227e-01, -5.5824e-01,
                        -9.9064e-04]],
              
                      [[-4.6138e-01, -2.6536e-01, -1.5010e-02, -5.8349e-01,  2.3662e-01,
                         6.4257e-01, -6.0409e-01,  2.2543e-01,  1.5230e-01,  2.5141e-01,
                         5.7945e-01]],
              
                      [[ 4.7529e-01,  9.3493e-02,  4.4330e-01,  1.0722e+00, -2.1116e-01,
                        -4.0567e-01,  1.4283e-01, -5.5884e-02,  4.8028e-01,  1.7262e-01,
                        -9.6913e-02]],
              
      

# clear GPU cache

In [23]:
with torch.no_grad():
    torch.cuda.empty_cache()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Debug

In [ ]:
from data_reader import CINC2021Reader as DR
from dataset import CINC2021 as DG

In [ ]:
?DG

In [ ]:
dg = DG(TrainCfg)

In [ ]:
dg._check_nan()

In [ ]:
hehe_data = dg.reader.load_data("JS35065")

In [ ]:
plt.plot(hehe_data[11])

In [ ]:
from utils.scoring_aux_data import *

%load_ext autoreload
%autoreload 2

In [ ]:
dx_mapping_scored

In [ ]:
dx_mapping_all.columns.values

In [ ]:
equiv_class_dict

In [ ]:
dx_mapping_scored

In [ ]:
equiv_class_dict

In [ ]:
from data_reader import CINC2021Reader as DR

%load_ext autoreload
%autoreload 2

In [ ]:
dr = DR("/home/wenh06/Jupyter/data/CinC2021/")

In [ ]:
dr._compute_cooccurrence()

In [ ]:
dr.load_ann(dr.all_records.G[10420], raw=True)

In [ ]:
dr.load_ann(dr.all_records.G[332], raw=True)

In [ ]:
dr.all_records.G[333]

In [ ]:
dr.all_records.G[3139]

In [ ]:
251238007, 6180003